In [2]:
from PIL import Image
import numpy as np
import pandas as pd
import os

In [47]:
class RunningStats:

    def __init__(self, ratio, resize= (240,240)):
        self.n = 0
        self.old_m = 0
        self.new_m = 0
        self.old_s = 0
        self.new_s = 0
        self.resize = resize
        self.ratio = ratio

    def clear(self):
        self.n = 0

    def push(self, x):
        self.n += 1
        
        img = Image.open(x)
        img = img.resize(self.resize)
        img = np.array(img, dtype=np.float)
        image = img * self.ratio
        image = image.reshape(self.resize[0] * self.resize[1])
#         print(image)

        if self.n == 1:
            self.old_m = self.new_m = np.mean(image)
            self.old_s = 0
        else:
            self.new_m = self.old_m + (np.mean(image) - self.old_m) / self.n
            self.new_s = self.old_s + (np.std(image) - self.old_m) * (np.std(image) - self.new_m)

            self.old_m = self.new_m
            self.old_s = self.new_s

    def mean(self):
        return self.new_m if self.n else 0.0

    def variance(self):
        return self.new_s / (self.n - 1) if self.n > 1 else 0.0

    def standard_deviation(self):
        return np.sqrt(self.variance())

In [4]:
train_csv = '/Users/brianrodriguez/Development/LenlessCameraML/data_csv/kfold_train.csv'
test_csv = '/Users/brianrodriguez/Development/LenlessCameraML/data_csv/kfold_test.csv'
root_dir = '/Volumes/MyPassport/ML/Deep_Learning/lensless_image_data/'
pd_csv = pd.read_csv(train_csv)

In [30]:
new_max = 0
resize = (240,240)
for i in pd_csv.itertuples():
    path = os.path.join(root_dir, i[1])
    image = Image.open(path)
    image = image.resize(resize)
    image = np.asarray(image)
    image = image.reshape(resize[0] * resize[1])
    im_max = image.max()
    if im_max > new_max:
        new_max = im_max
        print(i)
new_max

Pandas(Index=0, File='0/0.png', Label=0)
Pandas(Index=1, File='0/2.png', Label=0)
Pandas(Index=3, File='0/4.png', Label=0)
Pandas(Index=5, File='0/7.png', Label=0)


65535

In [54]:
pd_csv = pd.read_csv(test_csv)
for i in pd_csv.itertuples():
    path = os.path.join(root_dir, i[1])
    image = Image.open(path)
    image = image.resize(resize)
    image = np.asarray(image)
    image = image.reshape(resize[0] * resize[1])
    im_max = image.max()
    if im_max > new_max:
        new_max = im_max
        print(i)
new_max

65535

In [56]:
len(pd_csv)

10800

In [55]:
ratio = 255/new_max
ratio

0.0038910505836575876

In [48]:
rs = RunningStats(ratio)

In [57]:
for i in pd_csv.itertuples():
    path = os.path.join(root_dir, i[1])
    rs.push(path)


In [58]:
print(rs.standard_deviation())
print(rs.mean())

139.86624980553643
157.1148628732102


In [1]:
class MaxNormalization(object):
    def __init__(self, max_val):
        self.max = max_val

    def __call__(self, img):
        norm_img = np.array(img, dtype= np.float) * self.max
        rows, cols = norm_img.shape
        norm_img = norm_img.reshape((cols, rows, 1))
        
        return norm_img

In [3]:
maxnorm = MaxNormalization(0.0038910505836575876)

In [5]:
path = os.path.join(root_dir, '0/7.png')
image = Image.open(path)

In [8]:
maxnorm(image).max()

255.0